In [1]:
from cryo_fts.lockin import LockinController
from cryo_fts.mirror import MirrorController
from cryo_fts.encoder import EncoderController
from cryo_fts.motor import MotorController
import time
import pandas as pd
#mirror = MirrorController()
lockin = LockinController(gpib_address=8)
encoder = EncoderController()
motor = MotorController()

Established connection to Prologix GPIB-USB Controller version 6.101 on COM5
Established connection to E201-9S V1.22 on COM4
Established connection to Device 1 SN: 83850 (X-LSM200A-SV2) -> Connection 1 (ASCII Serial port: COM3) on Connection 1 (ASCII Serial port: COM3)


In [ ]:
encoder.init()


In [4]:
motor.init()
motor.move_absolute(80, length_unit=None)

In [1]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p.device, p.description)

COM4 STMicroelectronics Virtual COM Port (COM4)
COM5 USB Serial Port (COM5)
COM3 USB Serial Port (COM3)


In [ ]:
#mirror.init()
mirror.scan_and_collect(velocity=10, sample_rate=5)
time.sleep(10) 
mirror.stop_scan()
mirror.close()

In [ ]:
lockin.init()

In [ ]:
freq = lockin.write('FREQ?', read=True)
print("Reference Frequency (Hz):", freq)

In [ ]:
lockin.get_x_y_r_theta()

In [ ]:
lockin.set_freq(900)
lockin.get_freq()

In [ ]:
lockin.get_amp()

In [ ]:
lockin.set_amp(0.0123)

In [ ]:
lockin.set_timeconstant(8)
lockin.get_timeconstant()

In [ ]:
lockin.set_sens(20)
lockin.get_sens()

In [ ]:
lockin.start_transmission(sample_rate= 10)
time.sleep(5)
lockin.stop_transmission()


In [ ]:
data = lockin.get_all()
print(data)

In [ ]:
encoder.start_transmission()
time.sleep(5)
encoder.stop_transmission()
encoder.get_latest()


In [ ]:
import numpy as np
import pandas as pd

data = np.load('../data/trial-run-diode.npz')
# df = pd.DataFrame('../data/trial-run-diode.npz')
R = data['R_V']

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(np.fft.fftshift(np.fft.fft(R - np.mean(R), len(R))))
# np.fft.fftfreq()
plt.show()

In [ ]:
fs = 1 / 0.1
N = len(R)
fft = np.fft.fft(R - np.mean(R))
freqs = np.fft.fftfreq(N, d = 0.1)
pos_freqs = freqs[freqs >= 0]
pos_fft = np.abs(fft[freqs >= 0])

bins = np.arange(0, fs/2, 0.1)
bin_freqs = 0.5*(bins[:-1] + bins[1:])
bin_fft = np.ones(len(bins)-1)

for i in range(len(bins)-1):
    bin_mask = (pos_freqs >= bins[i]) & (pos_freqs < bins[i+1])
    bin_fft[i] = np.mean(pos_fft[bin_mask]) 

plt.plot(bin_freqs, bin_fft)
plt.show()